In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
#依赖的包
from lib.dataloader import normal_and_generate_dataset_time, get_mask, get_adjacent, get_grid_node_map_maxtrix
#攻击函数import
from lib.utils_new import mask_loss, compute_loss, predict_and_evaluate, attack, random_attack, fgsm_attack, min_attack
from lib.utils_new import saliency, saliency_loss, attack_js, attack_js,attack_saliency,rand_attack
from model.GSNet import GSNet
from lib.early_stop import EarlyStopping
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import json
import configparser
import pickle as pkl
from time import time
from datetime import datetime
import shutil
import argparse
import random
import math

import sys
import os


In [ ]:

train_log_filename = "test_log_ch.txt"
train_log_filepath = os.path.join("./", train_log_filename)
now = datetime.now()
date_time = now.strftime("|%Y-%m-%d, %H:%M:%S| ")

curPath = os.path.abspath(os.path.dirname('__file__'))
rootPath = os.path.split(curPath)[0]
sys.path.append(rootPath)

In [ ]:

from mmcv import Config
cfgin = Config.fromfile('lib/attack_all_setting.yaml')



In [ ]:
if cfgin.dataset =='NYC':
    config_filename = 'config/nyc/GSNet_NYC_Config.json'
else: 
    config_filename = "config/chicago/GSNet_Chicago_Config.json"
with open(config_filename, 'r') as f:
    config = json.loads(f.read())

with open(train_log_filepath, "a") as f:
    f.write('测试文件运行时间：'+date_time)
    f.write('\r\n')
    f.write('超参数设置：')
    f.write('\r\n')
    f.write(json.dumps(config, sort_keys=True, indent=4))
    f.write('\r\n')
f.close
#GPU设置
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

north_south_map = config['north_south_map']
west_east_map = config['west_east_map']

all_data_filename = config['all_data_filename']
mask_filename = config['mask_filename']

road_adj_filename = config['road_adj_filename']
risk_adj_filename = config['risk_adj_filename']
poi_adj_filename = config['poi_adj_filename']
grid_node_filename = config['grid_node_filename']
grid_node_map = get_grid_node_map_maxtrix(grid_node_filename)
num_of_vertices = grid_node_map.shape[1]


patience = config['patience']
delta = config['delta']

if config['seed'] is not None:
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    np.random.seed(seed)
    random.seed(seed)


train_rate = config['train_rate']
valid_rate = config['valid_rate']

recent_prior = config['recent_prior']
week_prior = config['week_prior']
one_day_period = config['one_day_period']
days_of_week = config['days_of_week']
pre_len = config['pre_len']
seq_len = recent_prior + week_prior

training_epoch = config['training_epoch']


In [ ]:
batch_size = 32
learning_rate = config['learning_rate']

#模型结构
num_of_gru_layers = config['num_of_gru_layers']
gru_hidden_size = config['gru_hidden_size']
gcn_num_filter = config['gcn_num_filter']

loaders = []
scaler = ""
train_data_shape = ""
graph_feature_shape = ""
   ################## #########################
for idx, (x, y, target_times, high_x, high_y, high_target_times, scaler) in enumerate(normal_and_generate_dataset_time(
        all_data_filename,
        train_rate=train_rate,
        valid_rate=valid_rate,
        recent_prior=recent_prior,
        week_prior=week_prior,
        one_day_period=one_day_period,
        days_of_week=days_of_week,
        pre_len=pre_len)):
    if False:#前100个，没写默认是false
        x = x[:100]
        y = y[:100]
        target_times = target_times[:100]
        high_x = high_x[:100]
        high_y = high_y[:100]
        high_target_times = high_target_times[:100]

    if 'nyc' in all_data_filename:
        graph_x = x[:, :, [0, 46, 47], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))#4584*7*3*400
        high_graph_x = high_x[:, :, [0, 46, 47], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))#1337*7*3*400
        graph_x = np.dot(graph_x, grid_node_map)#4584*7*3*243
        high_graph_x = np.dot(high_graph_x, grid_node_map)
    if 'chicago' in all_data_filename:
        graph_x = x[:, :, [0, 39, 40], :, :].reshape(
            (x.shape[0], x.shape[1], -1, north_south_map*west_east_map))
        high_graph_x = high_x[:, :, [0, 39, 40], :, :].reshape(
            (high_x.shape[0], high_x.shape[1], -1, north_south_map*west_east_map))
        graph_x = np.dot(graph_x, grid_node_map)
        high_graph_x = np.dot(high_graph_x, grid_node_map)

    print("feature:", str(x.shape), "label:", str(y.shape), "time:", str(target_times.shape),
            "high feature:", str(high_x.shape), "high label:", str(high_y.shape))
    print("graph_x:", str(graph_x.shape),
            "high_graph_x:", str(high_graph_x.shape))
    if x.shape[0] ==1080:
        b = np.ones_like(x)
        b = np.where(x>1,1,0)

    if idx == 0:
        scaler = scaler
        train_data_shape = x.shape
        time_shape = target_times.shape
        graph_feature_shape = graph_x.shape
    loaders.append(Data.DataLoader(
        Data.TensorDataset(
            torch.from_numpy(x),
            torch.from_numpy(target_times),#4584*32
            torch.from_numpy(graph_x),
            torch.from_numpy(y)
        ),
        batch_size=batch_size,
        shuffle=(idx == 0)
    ))
    if idx == 2:
        high_test_loader = Data.DataLoader(
            Data.TensorDataset(
                torch.from_numpy(high_x),
                torch.from_numpy(high_target_times),
                torch.from_numpy(high_graph_x),
                torch.from_numpy(high_y)
            ),
            batch_size=batch_size,
            shuffle=(idx == 0)
        )
train_loader, val_loader, test_loader = loaders
################ ##################
nums_of_filter = []
for _ in range(2):
    nums_of_filter.append(gcn_num_filter)

GSNet_Model = GSNet(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
                    gru_hidden_size, time_shape[1], graph_feature_shape[2],
                    nums_of_filter, north_south_map, west_east_map)
print("模型参数---------------------------------------------")
print(train_data_shape[2], num_of_gru_layers, seq_len, pre_len,
        gru_hidden_size, time_shape[1], graph_feature_shape[2],
        nums_of_filter)

# multi gpu
if torch.cuda.device_count() > 10:
    print("Let's use", torch.cuda.device_count(), "GPUs!", flush=True)
    GSNet_Model = nn.DataParallel(GSNet_Model)
############### ######################
GSNet_Model.to(device)

GSNet_Model.load_state_dict(torch.load("data/model_chicago.pt"))


num_of_parameters = 0
for name, parameters in GSNet_Model.named_parameters():
    num_of_parameters += np.prod(parameters.shape)
print("Number of Parameters: {}".format(num_of_parameters), flush=True)

trainer = optim.Adam(GSNet_Model.parameters(), lr=learning_rate)
early_stop = EarlyStopping(patience=patience, delta=delta)

risk_mask = get_mask(mask_filename)
road_adj = get_adjacent(road_adj_filename)
risk_adj = get_adjacent(risk_adj_filename)
if poi_adj_filename == "":
    poi_adj = None
else:
    poi_adj = get_adjacent(poi_adj_filename)


In [ ]:

from lib.myfunction import get_root_logger, logger_info
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")


In [ ]:
torch.cuda.empty_cache()

In [ ]:

import yaml
from mmcv import Config
from lib.myfunction import log_test_results
from lib.outzoom import log_test_csv,node_map
from lib.outzoom import STZINB,min_impr,pgd_impr,random_impr,log_test_csv_time_K
from lib.myfunction import get_root_logger, logger_info
#配置logo
logger = get_root_logger('INFO', './logdata')
logger.info("Loading config file from ")

#根据配置选择攻击方法
if cfgin.attacker == 'white_attacker':
    header = ['time', 'epoch', 'dataset', 'model', 'node_select', 'method', 'K', 'batch size',
              'clean_RMSE', 'clean_recall', 'clean_MAP', 'clean_RCR',
              'adv_RMSE', 'adv_recall', 'adv_MAP', 'adv_RCR',
              'local_adv_RMSE', 'local_adv_recall', 'local_adv_MAP', 'local_adv_RCR',
              '00']

    file_name = 'result_chicago-data-c-{}_num-nodes{}_eps{}-model-{}'.format(
        cfgin.dataset, cfgin.test_attack_nodes, cfgin.test_epsilon, cfgin.backbone)
    log_test_results(cfgin.result_dir, header, file_name)

    for epoch in range(5):
        #break
        for out_K in cfgin.K:
            for i in cfgin.select_node:
                for j in cfgin.attack_method:
                    logger.info(i+'_'+j+"_"+str(out_K))
                    parm_att = j  
                    parm_node = i  

                    ack_map = node_map(i,12,out_K, 5,cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                       grid_node_map, device, data_type='nyc')
                    
                    adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                            grid_node_map, scaler, risk_mask, device, data_type='nyc')
                    
                    log_test_csv_time_K(epoch, out_K, adv_val_predict, val_target,
                                        val_predict, cfgin, i, j, file_name, risk_mask)
                    torch.cuda.empty_cache()
    for epoch in range(5):
        #break
        for out_K in [10,20,30,40]:
            if out_K == 40:
                a = 12
            else: a =47
            i = 'HGNS'
            j = 'STZINB'
            logger.info(i+'_'+j+"_"+str(out_K))
            parm_att = j  
            parm_node = i  

            ack_map = node_map(i, a, out_K, 5, cfgin, GSNet_Model, test_loader, risk_mask, road_adj, risk_adj, poi_adj,
                                grid_node_map, device, data_type='chicago')

            adv_val_predict, val_target, val_predict = eval("{0}".format(parm_att))(logger, cfgin, ack_map, GSNet_Model, test_loader,  road_adj, risk_adj, poi_adj,
                                                                                    grid_node_map, scaler, risk_mask, device, data_type='nyc')

            log_test_csv_time_K(epoch, out_K, adv_val_predict, val_target,
                                val_predict, cfgin, i, j, file_name, risk_mask)
            torch.cuda.empty_cache()
